# Binary Interfacing

Motivations of binary interfacing:

1. Use of binary libraries within the script language
1. From a C, C++ code, use a python interpreter and use flexibility of script languages.
 - AutoCAD vs List
 - Gimp vs Scheme and Python
 - Blender vs Python
 - Excell vs Visual Basic
 - ...
  Script language provides a powerful interface to Human to interact with a binary program, debuging, automating, macros etc.
  
## ctypes

It loads dynamic libraries built in C/C++ in python run-time and let all C/C++ types encapsulated in classes so that data can be interchanged.
It calls C/C++ functions within python.

class wrappers around  C values:
* `c_int, c_double, c_float, c_char`
* `c_int * n` for an integer array of size n.
  `int a[10];`     ->     `(c_int * 10)`
* `c_void_p` and `c_char_p`  for `void *` and `char *`
* use `POINTER(type)` for pointer to type
* `byref(value)` gives a pointer to value `POINTER(typeof(value))`

For structures. Inherit `ctypes.Structure` for creating a special wrapper:
```c
struct Complex {
    double x;
    double y;
}
```

```python
class Complex(ctypes.Structure):
    _fields_ = [("x", c_double), ("y", c_double)]
```

You need to set `func.restype` to set result type of Functions since python
does not have any idea about the function prototypes.
Also you can use `argtypes = (type1, type2, ...)` to set arguments type

```python
lib = ctypes.CDLL("binaryfile.so")
lib.func.restype = c_double

darray = c_double * 10
dval = darray( * [ 1, 2, 4, 5, 6, 7, 8, 9, 10, 3])  # * unwraps the list in set of arguments
lib.func( c_double(arg1), c_int(arg2), dval)
# or
lib.func.argtypes = c_double, c_int, darray
lib.func(arg1, arg2, dval )
         
```


In [1]:
from ctypes import *

# use system math library
mlib = CDLL('libm.so.6')

#this will return an integer since result type unknown
print(mlib.sin(c_double(3.1415926536)))

# try again by setting return type
mlib.sin.restype = c_double
print(mlib.sin(c_double(3.1415926536)))

0
-1.0206823934513925e-11


Following is a comprehensive example containing different parameter passing mechanisms in _[ctest.c](ctest.c)_
compile as 
> `gcc -shared -o libctest.so ctest.c`

to create `libctest.so`

In [2]:
# ctest.c contains following functions in C
# compile as 'gcc -shared -o libctest.so ctest.c' to create libctest.so
# struct Complex add(struct Complex , struct Complex );
# void swap(struct Complex *, struct Complex *);
# double sum(double f[], int n);
# void titlecase(char *);
# int tokenize(char sep, char *str, char t[][20]);
# void mult(double a[][100], double b[][100], double c[][100], int n, int r , int n);

# get the python library. searched in system path, specify full or 
# relative path if not in system library
lib = CDLL('./libctest.so')

class Complex(Structure):
    _fields_ = [("x", c_double), ("y", c_double)]
    
# struct Complex add(struct Complex , struct Complex );
lib.add.restype = Complex
r = lib.add(Complex(3.1, 4.2), Complex(1.9, 2.8))
print('1-add\n',r, r.x, r.y)

a = Complex(3, 4)
b = Complex(2, 7)

# void swap(struct Complex *, struct Complex *);
# send pointers to values to pass by pointer
lib.swap(byref(a), byref(b))
print('2-swap\n',a.x, a.y, b.x, b.y)

# double sum(double f[], int n);
# Array type. You can pass pointers as in C. BE CAREFULL ABOUT STORAGE
# YOU CAN GET A SEGFAULT in PYTHON!!!
lib.sum.restype = c_double
lib.sum.argtypes = POINTER(c_double), c_int 
myarr = (c_double * 10)( * [1, 2, 1, 6, 4, 2, 7, 3, 1, 5])
print('3-sum\n',lib.sum(myarr, 10))

# void titlecase(char *);
# c_char_p can be use to 0 terminated C strings. Make sure the storage is sufficient
# use value field to get the content
lib.titlecase.argtypes = (c_char_p,)
name = c_char_p(b'the advantages of script languages over OTHER languages')
lib.titlecase(name)
print('4-titlecase\n', name.value)

# int tokenize(char sep, char *str, char t[][20]);
# this is slightly tricky t needs to store resulting tokens, so need sufficient storage
lib.tokenize.restype = c_int
tokenstype = (c_char * 20) * 20
lib.tokenize.argtypes = c_char, c_char_p, tokenstype

# create an array for the tokenize result array of 20 strings
res = tokenstype ( )
for i in range(20):
    res[i] = create_string_buffer(20)
n = lib.tokenize(b' ', b'the advantages of script languages over OTHER languages', res )
tokens = [tok.value for tok in res[:n]]
print('5-tokenize\n',n,tokens)

# void mult(double a[][100], double b[][100], double c[][100], int n, int r , int n);
a = [[ 1, 2, 3, 4, 5], [11, 12, 13, 14, 15]]
b = [[1, 2], [2, 3],[4, 5], [6, 7],[8,9]]
arrtype = (c_double * 100) * 100

# initialize matrix a
matA = arrtype()
i = 0
for row in a:
    matA[i] = (c_double * 100)( * row)
    i += 1

# initialize matrix B
matB = arrtype()
i = 0
for row in b:
    matB[i] = (c_double * 100)( * row)
    i += 1
matC = arrtype()

# multiply
lib.mult.argtypes = arrtype, arrtype, arrtype, c_int, c_int, c_int
lib.mult(matA, matB, matC, 2, 5, 2)

# result
print('6-mult\n',[[matC[i][j] for i in range(2)] for j in range(2)])

OSError: ./libctest.so: cannot open shared object file: No such file or directory